In [1]:
import dill as pickle
import aesara.tensor as aet
import pandas as pd
import pycmtensor as cmt
from pycmtensor.pycmtensor import PyCMTensorModel, Beta, Weights
from pycmtensor.functions import logit, neg_loglikelihood
from pycmtensor.optimizers import Adam
from pycmtensor.results import Results

swissmetro = pd.read_csv("data/swissmetro.dat", sep="\t")
db = cmt.Database("swissmetro", swissmetro, choiceVar="CHOICE")
globals().update(db.variables)
# Removing some observations
exclude = ((PURPOSE != 1) * (PURPOSE != 3) + (CHOICE == 0)) > 0
db.remove(exclude)

# additional steps to format database
db.data["CHOICE"] -= 1 # set the first choice to 0
db.autoscale(variables=['TRAIN_CO', 'TRAIN_TT', 'CAR_CO', 'CAR_TT', 
    'SM_CO', 'SM_TT'], default=100., verbose=False)

In [2]:
class ResLogitLayer:
    def __init__(self, input, w_in, w_out, activation_in=None, activation_out=None):
        assert w_in.shape.eval()[1] == w_out.shape.eval()[0]
        if isinstance(input, (list, tuple)):
            assert len(input) == w_in.shape.eval()[0], f"index.0 of w_in must be of the same length as input"
            input = aet.concatenate(input, axis=1)

        assert isinstance(w_in, (Weights)), "w_in must be of type Weights"
        assert isinstance(w_out, (Weights)), "w_out must be of type Weights"
        self.w_in = w_in()
        self.w_out = w_out()
            
        h = aet.sigmoid(aet.dot(input, self.w_in))
        output = aet.sigmoid(aet.dot(h, self.w_out))
        self.input = input
        self.weights = [self.w_in, self.w_out]
        self.output = output + input

    
    def __repr__(self):
        return f"ResLogitLayer([{self.w_in.shape.eval()}, {self.w_out.shape.eval()}])"


class MNLmodel(PyCMTensorModel):
    def __init__(self, db):
        super().__init__(db)
        self.name = "myModel"
        self.inputs = db.tensors()  # keep track of inputs

        # declare model params here
        b_cost = Beta("b_cost", 0.0, None, None, 0)
        b_time = Beta("b_time", 0.0, None, None, 0)
        asc_train = Beta("asc_train", 0.0, None, None, 0)
        asc_car = Beta("asc_car", 0.0, None, None, 0)
        asc_sm = Beta("asc_sm", 0.0, None, None, 1)

        W1 = Weights("ResNet_01a", (3, 10), 0, True)
        W2 = Weights("ResNet_01b", (10, 3), 0, True)

        # append model params to self.params list
        self.append_to_params(locals())
        # self.append_to_params([b_cost, b_time, asc_train, asc_car, asc_sm, W1, W2])

        # Definition of the utility functions
        U_1 = b_cost * db["TRAIN_CO"] + b_time * db["TRAIN_TT"] + asc_train
        U_2 = b_cost * db["SM_CO"] + b_time * db["SM_TT"] + asc_sm
        U_3 = b_cost * db["CAR_CO"] + b_time * db["CAR_TT"] + asc_car
        U = [U_1, U_2, U_3]
        rh = ResLogitLayer(U, W1, W2)

        # definition of the choice output
        self.y = db["CHOICE"]

        # symbolic expression for the choice model
        self.p_y_given_x = logit(rh.output, [db["TRAIN_AV"], db["SM_AV"], db["CAR_AV"]])

        # declare Regularizers here:
        # L1 regularization cost
        self.L1 = abs(b_cost()) + abs(b_time())

        # L2 regularization cost
        self.L2 = b_cost() ** 2 + b_time() ** 2

        # symbolic expression for the cost fuction
        self.cost = neg_loglikelihood(self.p_y_given_x, self.y)
        self.cost = self.cost

        # symbolic description of how to compute prediction as class whose
        # probability is maximal
        self.pred = aet.argmax(self.p_y_given_x, axis=1)

In [ ]:
# train function
model = cmt.train(MNLmodel, database=db, optimizer=Adam, batch_size=256, lr_init=0.01, max_epoch=10)
        
with open("myModel.pkl", "rb") as f:
    model = pickle.load(f)

result = Results(model, db, show_weights=True)

In [6]:
model = MNLmodel(db)
model.__dict__

ResNet_01b([10  3])
